# Steps: 
1. Determine which variables to use
2. Cut down size of dataframe (maybe top 10 and do dummies? or an aggregate (# of pitbulls per month)
3. Create multivariate dataframe, including timestamp index
4. Scale, series_to_supervise
5. Fit data, split to train_test sets
6. Use univariate LSTM, tune to lowest MSE
7. Compare univariate to multivariate scores, make analysis
8. Save model and fit into function
9. Write new prediction function? 
10. Conclude, interpret
11. Make more visualizations
12. Blog about it
13. Where do we go from here? 
14. What is the "so what"? 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/raw_data.csv')

In [4]:
df.set_index('ValidDate', inplace = True)

In [10]:
df = df.drop(['ExpYear'], axis = 1)

In [11]:
df.head()

,LicenseType,Breed,Color,DogName,OwnerZip
ValidDate,,,,,
2014-12-02 09:40:53,Dog Individual Neutered Male,COCKAPOO,BROWN,CHARLEY,15236
2014-12-02 09:45:25,Dog Senior Citizen or Disability Neutered Male,GER SHEPHERD,BLACK/BROWN,TACODA,15238
2014-12-02 09:47:55,Dog Individual Spayed Female,GER SHEPHERD,BLACK,CHARLY,15205
2014-12-02 10:02:33,Dog Individual Spayed Female,LABRADOR RETRIEVER,BLACK,ABBEY,15143
2014-12-02 10:05:50,Dog Individual Female,GER SHORTHAIR POINT,BROWN,CHARLEY,15228


In [14]:
df['Breed'].value_counts().sum()

286724

In [ ]:
#I would definitely need to look at cutting that number down- that would be way too noisy.
#My initial thought is to see what the top 10 breeds are
#Maybe from there I can either to dummies or aggregate

In [46]:
top_ten = df['Breed'].value_counts()[:11]

In [47]:
top_ten
#bar chart here

MIXED                  29009
LABRADOR RETRIEVER     19713
LAB MIX                17714
GOLDEN RETRIEVER        9344
GER SHEPHERD            8437
SHIH TZU                7976
BEAGLE                  7960
CHIHUAHUA               7664
TAG                     7475
AM PIT BULL TERRIER     7332
YORKSHIRE TERRIER       6268
Name: Breed, dtype: int64

In [ ]:
#TAG is not a type of dog. they are denoting that they are putting tags on an existing dog

In [67]:
tag_df = df.loc[df['Breed'] == 'TAG']
tag_df.loc[(tag_df['DogName'] == 'SHADOW') & (tag_df['OwnerZip'] == 15102)]

,LicenseType,Breed,Color,DogName,OwnerZip
ValidDate,,,,,
2015-03-12 11:11:36,Dog Individual Spayed Female,TAG,BLACK,SHADOW,15102
2017-01-10 09:39:46,Dog Individual Spayed Female,TAG,BLACK,SHADOW,15102
2015-12-11 10:35:08,Dog Individual Spayed Female,TAG,BLACK,SHADOW,15102


In [61]:
a = len(tag_df['DogName'])
b = len(tag_df['DogName'].value_counts())
a-b

5708

In [68]:
top_ten_df = df[(df['Breed'] == 'MIXED') |
                (df['Breed'] == 'LABRADOR RETRIEVER') |
                (df['Breed'] == 'LAB MIX') |
                (df['Breed'] == 'GOLDEN RETRIEVER') |
                (df['Breed'] == 'GER SHEPHERD') |
                (df['Breed'] == 'SHIH TZU') |
                (df['Breed'] == 'BEAGLE') |
                (df['Breed'] == 'CHIHUAHUA') |
                (df['Breed'] == 'YORKSHIRE TERRIER') |
                (df['Breed'] == 'AM PIT BULL TERRIER')]

In [ ]:
#In the univariate model, we summed the total licenses per day. I think we could sum each breed per day,
#and also include a column that shows the % of dogs per day that were top 10
#Let's start by summing the top 10 per day